## CRF model with cross validation

Modelo CRF para ato de aviso de licitação (com dados rotulados manualmente)

In [1]:
# !pip install sklearn_crfsuite
# !pip install nltk

import pandas as pd
import sklearn_crfsuite
import nltk
from nltk.tokenize import word_tokenize

In [2]:
path_data_train = pd.read_parquet('./result/test.parquet')
path_data_test = pd.read_parquet('./result/test.parquet')

data_train = pd.DataFrame(path_data_train)
data_test = pd.DataFrame(path_data_test)


In [28]:
data_test.head()

,ORGAO_LICITANTE,NUM_LICITACAO,OBJ_LICITACAO,MODALIDADE_LICITACAO,PROCESSO,DATA_ABERTURA,TIPO_OBJ,VALOR_ESTIMADO,CODIGO_SISTEMA_COMPRAS,SISTEMA_COMPRAS,treated_text,IOB
0,None,249/00,AQUISIÇÃO DE MATERIAL DE CONSUMO ; MATERIAL PA...,CONVITE,None,24/07/00,AQUISIÇÃO,None,None,http://www fazenda df gov.br,CONVITE Nº 249/00 Objeto: AQUISIÇÃO DE MATERIA...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
1,None,1.437/2008,Contratação de empresa para prestação de servi...,PREGÃO ELETRÔNICO,053.001.258/2008,16 de janeiro de 2009,prestação de serviços,None,None,www.compras.df.gov.br,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 1.437/...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
2,None,5/2003,"Aquisição de material de consumo ( limpeza, co...",CONCORRENCIA,None,10.03.2003,None,None,None,www.fazenda.df.gov.br,AVISOS DE LICITAÇÃO CONCORRENCIA N.º 5/2003-Su...,O O O B-MODALIDADE_LICITACAO O O B-NUM_LICITAC...
3,None,50/2015,Aquisição de material consumo Termoplástico Te...,PREGÃO ELETRÔNICO,113.013360/2015,27 de janeiro de 2016,Aquisição,None,None,None,AVISO DE LICITAÇÃ NOVA DATA PREGÃO ELETRÔNICO ...,O O O O O B-MODALIDADE_LICITACAO I-MODALIDADE_...
4,None,964/2008,Contratação de firma especializada em confecçã...,PREGÃO ELETRÔNICO,220.000.570/2008,11 de setembro de 2008,Contratação,None,None,www.compras.df.gov.br,PREGÃO ELETRÔNICO Nº 964/2008. Objeto: Contrat...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...


Load csv with text and labels

In [5]:
# tratando dados de treino
 
from nltk.tokenize import RegexpTokenizer

TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize

x_train = []
y_train = []
for row in range(len(data_train)):
    if pd.notna(data_train['treated_text'][row]):
        x_train.append(TOKENIZER(data_train['treated_text'][row]))
        y_train.append(data_train['IOB'][row].split())
len(x_train), len(y_train)

(71, 71)

In [6]:
# tratando dados de teste

from nltk.tokenize import RegexpTokenizer

TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize

x_test = []
y_test = []
for row in range(len(data_test)):
    if pd.notna(data_test['treated_text'][row]):
        x_test.append(TOKENIZER(data_test['treated_text'][row]))
        y_test.append(data_test['IOB'][row].split())
len(x_test), len(y_test)

(71, 71)

Create dictionary feature for each word in each sequence in x

In [7]:
def get_features(sentence):
    """Create features for each word in act.
    Create a list of dict of words features to be used in the predictor module.
    Args:
        act (list): List of words in an act.
    Returns:
        A list with a dictionary of features for each of the words.
    """
    sent_features = []
    for i in range(len(sentence)):
        word_feat = {
            'word': sentence[i].lower(),
            'word[-3:]': sentence[i][-3:],
            'word[-2:]': sentence[i][-2:],
            'capital_letter': sentence[i][0].isupper(),
            'word_istitle': sentence[i].istitle(),
            'all_capital': sentence[i].isupper(),
            'word_isdigit': sentence[i].isdigit(),
            # Uma palavra antes
            'word_before': '' if i == 0 else sentence[i-1].lower(),
            'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
            'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
            'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
            # Duas palavras antes
            'word_before2': '' if i in [0, 1] else sentence[i-2].lower(),
            'word_before_isdigit2': '' if i in [0, 1] else sentence[i-1].isdigit(),
            'word_before_isupper2': '' if i in [0, 1] else sentence[i-1].isupper(),
            'word_before_istitle2': '' if i in [0, 1] else sentence[i-1].istitle(),
            # Uma palavra depois
            'word_after': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
            'word_after_isdigit': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
            'word_after_isupper': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
            'word_after_istitle': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),
            # Duas palavras depois
            'word_after2': '' if i+2 >= len(sentence) else sentence[i+2].lower(),
            'word_after_isdigit2': '' if i+2 >= len(sentence) else sentence[i+2].isdigit(),
            'word_after_isupper2': '' if i+2 >= len(sentence) else sentence[i+2].isupper(),
            'word_after_istitle2': '' if i+2 >= len(sentence) else sentence[i+2].istitle(),
            # 'word_before': '' if i == 0 else sentence[i-1].lower(),
            # 'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
            # 'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
            # 'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
            # 'word_after:': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
            # 'word_after_isdigit:': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
            # 'word_after_isupper:': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
            # 'word_after_istitle:': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),
            'BOS': i == 0,
            'EOS': i == len(sentence)-1
        }
        sent_features.append(word_feat)
    return sent_features

In [8]:
for i in range(len(x_train)):
    x_train[i] = get_features(x_train[i])

for i in range(len(x_test)):
    x_test[i] = get_features(x_test[i])

Separate train and test splits (in order)

In [9]:
x_train[0]

[{'word': 'convite',
  'word[-3:]': 'ITE',
  'word[-2:]': 'TE',
  'capital_letter': True,
  'word_istitle': False,
  'all_capital': True,
  'word_isdigit': False,
  'word_before': '',
  'word_before_isdigit': '',
  'word_before_isupper': '',
  'word_before_istitle': '',
  'word_before2': '',
  'word_before_isdigit2': '',
  'word_before_isupper2': '',
  'word_before_istitle2': '',
  'word_after': 'nº',
  'word_after_isdigit': False,
  'word_after_isupper': False,
  'word_after_istitle': True,
  'word_after2': '249',
  'word_after_isdigit2': True,
  'word_after_isupper2': False,
  'word_after_istitle2': False,
  'BOS': True,
  'EOS': False},
 {'word': 'nº',
  'word[-3:]': 'Nº',
  'word[-2:]': 'Nº',
  'capital_letter': True,
  'word_istitle': True,
  'all_capital': False,
  'word_isdigit': False,
  'word_before': 'convite',
  'word_before_isdigit': False,
  'word_before_isupper': True,
  'word_before_istitle': False,
  'word_before2': '',
  'word_before_isdigit2': '',
  'word_before_isupp

In [10]:
# !pip install scipy
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

rs = RandomizedSearchCV(crf, params_space, 
                        cv=5, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50)
rs.fit(x_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_p

RandomizedSearchCV(cv=5,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5e14f50430>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5e15a5f550>},
                   verbose=1)

In [11]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.31932097048088937, 'c2': 0.009530356867673853}
best CV score: 0.8764664602201394
model size: 0.15M


In [ ]:
# !pip install matplotlib
# import matplotlib.pyplot as plt
# from sklearn.model_selection import RandomizedSearchCV
# plt.style.use('ggplot')

# _x = rs.cv_results_['param_c1']
# _y = rs.cv_results_['param_c2']
# _c = rs.cv_results_['mean_test_score']

# fig = plt.figure()
# fig.set_size_inches(12, 12)
# ax = plt.gca()
# ax.set_yscale('log')
# ax.set_xscale('log')
# ax.set_xlabel('C1')
# ax.set_ylabel('C2')
# ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
#     min(_c), max(_c)
# ))

# ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

# print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

In [12]:
crf = rs.best_estimator_

classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3
))

/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-MODALIDADE_LICITACAO', 'B-NUM_LICITACAO', 'I-NUM_LICITACAO', 'B-OBJ_LICITACAO', 'I-OBJ_LICITACAO', 'B-DATA_ABERTURA', 'I-DATA_ABERTURA', 'B-SISTEMA_COMPRAS', 'I-SISTEMA_COMPRAS', 'I-MODALIDADE_LICITACAO', 'B-PROCESSO', 'I-PROCESSO', 'B-CODIGO_SISTEMA_COMPRAS', 'B-VALOR_ESTIMADO', 'I-VALOR_ESTIMADO', 'I-CODIGO_SISTEMA_COMPRAS', 'B-TIPO_OBJ', 'B-ORGAO_LICITANTE', 'I-ORGAO_LICITANTE'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


                          precision    recall  f1-score   support

  B-MODALIDADE_LICITACAO      1.000     1.000     1.000        72
         B-NUM_LICITACAO      0.988     0.988     0.988        80
         I-NUM_LICITACAO      0.973     1.000     0.986        73
         B-OBJ_LICITACAO      1.000     1.000     1.000        70
         I-OBJ_LICITACAO      0.984     1.000     0.992      2053
         B-DATA_ABERTURA      1.000     0.947     0.973        75
         I-DATA_ABERTURA      0.985     1.000     0.993       200
       B-SISTEMA_COMPRAS      1.000     0.983     0.992        60
       I-SISTEMA_COMPRAS      1.000     1.000     1.000       185
  I-MODALIDADE_LICITACAO      1.000     1.000     1.000        73
              B-PROCESSO      1.000     1.000     1.000        59
              I-PROCESSO      1.000     1.000     1.000       160
B-CODIGO_SISTEMA_COMPRAS      1.000     1.000     1.000        12
        B-VALOR_ESTIMADO      1.000     0.952     0.976        21
        I

In [13]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
        

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-OBJ_LICITACAO -> I-OBJ_LICITACAO 5.779369
O      -> O       5.336256
I-PROCESSO -> I-PROCESSO 5.218414
B-CODIGO_SISTEMA_COMPRAS -> I-CODIGO_SISTEMA_COMPRAS 4.716489
I-SISTEMA_COMPRAS -> I-SISTEMA_COMPRAS 4.555286
I-VALOR_ESTIMADO -> I-VALOR_ESTIMADO 4.056591
I-DATA_ABERTURA -> I-DATA_ABERTURA 3.951341
B-VALOR_ESTIMADO -> I-VALOR_ESTIMADO 3.747090
B-NUM_LICITACAO -> I-NUM_LICITACAO 3.729454
I-ORGAO_LICITANTE -> I-ORGAO_LICITANTE 3.462145
B-MODALIDADE_LICITACAO -> I-MODALIDADE_LICITACAO 3.349387
I-MODALIDADE_LICITACAO -> I-MODALIDADE_LICITACAO 3.263376
B-PROCESSO -> I-PROCESSO 3.170532
B-DATA_ABERTURA -> I-DATA_ABERTURA 2.972631
B-ORGAO_LICITANTE -> I-ORGAO_LICITANTE 2.635686
B-SISTEMA_COMPRAS -> I-SISTEMA_COMPRAS 2.431726
B-OBJ_LICITACAO -> I-OBJ_LICITACAO 2.410537
O      -> B-SISTEMA_COMPRAS 2.211252
O      -> B-DATA_ABERTURA 2.204350
O      -> B-NUM_LICITACAO 1.926168

Top unlikely transitions:
B-DATA_ABERTURA -> B-DATA_ABERTURA -0.673903
B-VALOR_ESTIMADO -> 

In [14]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
6.654827 B-CODIGO_SISTEMA_COMPRAS word_before:uasg
4.227421 O        word:objeto
3.829761 O        word:200
3.796002 O        word_before2:gov
3.664366 O        word_before:2008
3.545483 B-VALOR_ESTIMADO word_after:851
3.464869 B-OBJ_LICITACAO word_before:objeto
3.380213 I-NUM_LICITACAO word_before2:nº
3.372236 O        word_after2:reais
3.324524 B-NUM_LICITACAO word_after2:2000
3.266243 B-PROCESSO word:060
3.266243 B-PROCESSO word[-3:]:060
3.243848 O        word:valor
3.109380 B-MODALIDADE_LICITACAO word_before:licitação
3.108587 B-ORGAO_LICITANTE word:administração
3.097680 O        word:data
2.981170 I-DATA_ABERTURA word_after2:às
2.948488 I-SISTEMA_COMPRAS word_before:http
2.861517 O        word[-2:]:um
2.824516 B-VALOR_ESTIMADO word_before:01
2.798009 B-DATA_ABERTURA word_before:abertura
2.778711 B-DATA_ABERTURA word_after2:cópia
2.772327 B-ORGAO_LICITANTE word_before:a
2.758218 I-CODIGO_SISTEMA_COMPRAS word_before:uasg
2.721328 I-MODALIDADE_LICITACAO word_after2:57
